In [ ]:
!pip install -U git+https://github.com/huggingface/diffusers
!pip install -q controlnet-aux transformers accelerate peft gradio==3.50.2 huggingface_hub safetensors
!pip install -q -U xformers --index-url https://download.pytorch.org/whl/cu121 --no-dependencies

from diffusers import StableDiffusionControlNetPipeline, UNet2DConditionModel, ControlNetModel, DDIMScheduler
from huggingface_hub import hf_hub_download
import torch
import cv2
import numpy as np
from PIL import Image
import gradio as gr


In [ ]:
hyper_sd = hf_hub_download("ByteDance/Hyper-SD", "Hyper-SD15-2steps-lora.safetensors")
sd_model = hf_hub_download("Yntec/epiCPhotoGasm", "epiCPhotoGasmVAE.safetensors")

controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-scribble", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_single_file(
    sd_model,
    use_safetensors=True,
    controlnet=controlnet,
    torch_dtype=torch.float16
).to("cuda")
pipe.load_lora_weights(hyper_sd)
pipe.fuse_lora()
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

In [ ]:
image_size = (512, 768)

def generate_canny(input_image):
    input_image = np.array(input_image)

    input_image = cv2.resize(input_image, image_size)
    input_image = 255 - cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    canny_image = cv2.ximgproc.thinning(input_image)
    canny_image = canny_image[:, :, None]
    canny_image = np.concatenate([canny_image, canny_image, canny_image], axis=2)
    canny_image = Image.fromarray(canny_image)
    return canny_image

def generate(prompt, negative_prompt, seed, eta, controlnet_conditioning_scale, scale, num_inference_steps, guidance_scale, canny_image):
    generator = torch.manual_seed(0)
    image = pipe(
        prompt, negative_prompt=negative_prompt,
        image=canny_image,
        generator=generator,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        eta=eta,
        controlnet_conditioning_scale=float(controlnet_conditioning_scale),
        cross_attention_kwargs={"scale": float(scale)},
    ).images[0]
    config = " | ".join([
        f"seed: {seed}",
        f"eta: {eta}",
        f"prompt: {prompt}",
        f"negative_prompt: {negative_prompt}",
        f"controlnet_conditioning_scale: {controlnet_conditioning_scale}",
        f"scale: {scale}",
        f"num_inference_steps: {num_inference_steps}",
        f"guidance_scale: {guidance_scale}"
    ])

    temp_image_path = "temp1.jpg"
    im = image.resize(image_size)
    im.save(temp_image_path, quality=95)
    return temp_image_path, config

with gr.Blocks(title=f"Realtime Latent Consistency Model") as demo:
    with gr.Box():
      with gr.Row():
        with gr.Column():
            with gr.Row():
              prompt = gr.Textbox(show_label=False, value="1girl red dress")
            with gr.Row():
              negative_prompt = gr.Textbox(show_label=False, value="blurry, text")
        with gr.Column():
            with gr.Row():
              scale = gr.Slider(minimum=0, maximum=1, step=0.1, value=1, label="lora_scale")
              num_inference_steps = gr.Slider(minimum=1, maximum=50, step=1, value=2, label="num_inference_steps")
              eta = gr.Slider(minimum=0, maximum=1, step=0.1, value=1, label="eta (lower for HQ + more steps)")
            with gr.Row():
              seed = gr.Slider(minimum=1, maximum=10000000, step=1, value=78901234, label="seed")
              guidance_scale = gr.Slider(minimum=0, maximum=10, step=0.2, value=1, label="guidance_scale")
              controlnet_conditioning_scale = gr.Slider(minimum=0, maximum=1, step=0.1, value=0.5, label="controlnet_scale")
    with gr.Row():
        input_image = gr.Image(
            show_label=False,
            type="pil",
            tool="color-sketch",
            source="canvas",
            width=image_size[0],
            height=image_size[1],
            brush_radius=5.0,
            interactive=True,
        )
        canny_image = gr.Image(
            show_label=False,
            type="pil",
            tool="color-sketch",
            source="upload",
            interactive=True,
            width=image_size[0],
            height=image_size[1],
        )
        output_image = gr.Image(
            show_label=False,
            type="pil",
            tool="color-sketch",
            source="upload",
            interactive=False,
            width=image_size[0],
            height=image_size[1],
        )
    with gr.Row():
      config = gr.Label(show_label=False)

    input_image.change(fn=generate_canny, inputs=[input_image], outputs=[canny_image], show_progress=False)
    for widget in (prompt, negative_prompt, seed, eta, controlnet_conditioning_scale, scale, num_inference_steps, guidance_scale, canny_image):
      widget.change(
          fn=generate,
          inputs=[prompt, negative_prompt, seed, eta, controlnet_conditioning_scale, scale, num_inference_steps, guidance_scale, canny_image],
          outputs=[output_image, config],
          show_progress=False
      )

demo.launch(inline=False, share=True, debug=True)